# Alzheimer's Disease classification from anatomical MRI

### This notebook explores the use of a low-dimensional feature space to classify Alzheimer's disease from anatomical MRI images.
 
Briefly, the pipeline involves the following steps and technical features:

- Data formating and quality check
- Feature extraction and visualization
- Synthetic class balancing
- Random forest classification as a function of features (principal components) used
- Permutation testing for statistical significance (parallel processing)
- Feature interpretation

### Import analysis and plotting libraries

In [0]:
# "standard"
import numpy as np
import pandas as pd

# machine learning and statistics
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from scipy.stats import false_discovery_control

# Parallel computing
import dask
from dask.distributed import Client, progress

# plotting
import matplotlib.pyplot as plt
import seaborn as sns

# misc
import cv2
import magic
from IPython.display import clear_output

## Apply random forest classifier to first two PCs

In [0]:
# Train random forest classifier and apply to test data
## for now again only with first two PCs

rf_1 = RandomForestClassifier()

# Train classifier
load_train = np.matmul(train_data, loadings[:,:2])
rf_1.fit(load_train, train_lab_idx)

# Properly format test data and extract same features (PC loadings)
test_data = []
for i in range(len(test)):
    test_data.append(np.reshape(test.iloc[i].img_arr, (1, -1)))    
test_data = np.vstack(test_data[:])

load_test = np.matmul(test_data, loadings[:,:2])

# Predict unseen data
predictions_1 = rf_1.predict(load_test)
test_lab_idx = np.asarray(test.iloc[:].label)

# compare predictions with test labels and compute accuracy
result = predictions_1 - test_lab_idx
result_binary = np.argwhere(result == 0)
correct = np.size(result_binary,0)
acc = correct/test_lab_idx.shape[0]*100
print(acc)

#### When using the top two PCs, we see a modest classification rate on the test set. However, using two PC's here is arbitrary and is easy simply due to visualization purposes. We can also exmaine accuracy as a function of the number of PCs and visualize the PC loading maps to interpret the classification results. But first, let's run some statistics to determine if the classification is significant in its current state.

## Permutation testing on shuffled data as a control

### Parallelize time-consuming iterative processes with Dask (local)

In [0]:
client = Client(threads_per_worker = 4, n_workers = 1)
client

### Shuffle data labels in the training set when building RF classifier

#### Do this a number of times to generate a null distribution

In [0]:
test_lab_idx = np.asarray(test.iloc[:].label)
train_lab_idx_shuffle = train_lab_idx
load_train = np.matmul(train_data, loadings[:,:2])

rng = np.random.default_rng()

def classification_RF_shuffle(load_train, train_lab_idx_shuffle, load_test, test_lab_idx):
    
    # Train classifier with new shuffled labels each iteration
    rf_1 = RandomForestClassifier()
    rng.shuffle(train_lab_idx_shuffle)
    rf_1.fit(load_train, train_lab_idx_shuffle)

    # Predict unseen data
    predictions_1 = rf_1.predict(load_test)
    test_lab_idx = np.asarray(test.iloc[:].label)

    # compare predictions with test labels and compute accuracy
    result = predictions_1 - test_lab_idx
    result_binary = np.argwhere(result == 0)
    correct = np.size(result_binary,0)
    acc = correct/test_lab_idx.shape[0]*100
    return acc

# Parallelize lazy function to reduce computation time
## limit number of iterations in this function to account for computer memory limit
def iteration_permute(load_train, train_lab_idx_shuffle, load_test, test_lab_idx):
    
    acc_shuffle = []
    for i in range(100):
        acc_tmp = dask.delayed(classification_RF_shuffle)(load_train, train_lab_idx_shuffle, load_test, test_lab_idx)
        acc_shuffle.append(acc_tmp)
    
    # compute values
    acc_shuffle = dask.compute(*acc_shuffle)
    acc_shuffle = np.array([float(num) for num in acc_shuffle])
    clear_output(wait = True)
    return acc_shuffle

## Permute a sufficient number of times (here 500)

### Not the most efficient way to do this, but must consider memory on local machine

In [0]:
%%time

acc_shuffle_compile = []
for i in range(5):
    acc_tmp = iteration_permute(load_train, train_lab_idx_shuffle, load_test, test_lab_idx)
    acc_shuffle_compile = np.concatenate((acc_shuffle_compile, acc_tmp), axis=0)

b = plt.boxplot(acc_shuffle_compile)
plt.title('Shuffled Classification Accuracy')
plt.xlabel('Shuffled')
plt.ylabel('Accuracy (%)')
plt.ylim(30, 60)

## Side note: validate parallel processing by comparing computation against serial processing

In [0]:
%%time

# simple test to ensure that parallel processing is faster

# traditional for loop
acc_shuffle_nodask = []
for i in range(100):
    acc_tmp = classification_RF_shuffle(load_train, train_lab_idx, load_test, test_lab_idx)
    acc_shuffle_nodask.append(acc_tmp)

In [0]:
%%time

# dask parallelization
acc_shuffle_dask = []
for i in range(1):
    acc_tmp = iteration_permute(load_train, train_lab_idx_shuffle, load_test, test_lab_idx)
    acc_shuffle_dask = np.concatenate((acc_shuffle_dask, acc_tmp), axis=0)
    clear_output(wait = True)

#### We can in fact see that parallelization with dask does markedly reduce the computation time (on the CPU). Of course, IO data transfer is the bottleneck here, but this will reduce computation time enough for this example on a Jupyter Notebook.

## Determine significance of classification accuracy (permutation test)

In [0]:
null_above_value = (acc_shuffle_compile > acc).sum()
p_value = null_above_value/np.size(acc_shuffle_compile,0)
print(p_value)

# We can see here that the classification accuracy is not sigificcantly different from chance with p-value > 0.05

## Now explore RF classification as a function of PCs

In [0]:
def classification_RF_PCs(train_data, train_lab_idx, test_data, test_lab_idx, loadings, num_pc):

    load_train = np.matmul(train_data, loadings[:,:num_pc])
    
    rf_1 = RandomForestClassifier()
    rf_1.fit(load_train, train_lab_idx)
    
    load_test = np.matmul(test_data, loadings[:,:num_pc])
    
    # Predict unseen data
    predictions_1 = rf_1.predict(load_test)
    test_lab_idx = np.asarray(test.iloc[:].label)
    
    # compare predictions with test labels and compute accuracy
    result = predictions_1 - test_lab_idx
    result_binary = np.argwhere(result == 0)
    correct = np.size(result_binary,0)
    acc = correct/test_lab_idx.shape[0]*100
    return acc

def iteration_PCs(train_data, train_lab_idx, test_data, test_lab_idx, loadings, pc_idx):
    
    acc_PC = []
    for i in pc_idx:
        acc_tmp = dask.delayed(classification_RF_PCs)(train_data, train_lab_idx, test_data, test_lab_idx, loadings, i)
        acc_PC.append(acc_tmp)
    
    # compute values
    acc_PC = dask.compute(*acc_PC)
    clear_output(wait = True)
    acc_PC = np.array([float(num) for num in acc_PC])
    return acc_PC

### Batch parallel processing to conserve memory

In [0]:
%%time

idx = [np.linspace(1,50,50, dtype = 'int'), np.linspace(51,100,50, dtype = 'int')]

acc_PC_compile = []
for i in range(len(idx)):
    
    acc_tmp = iteration_PCs(train_data, train_lab_idx, test_data, test_lab_idx, loadings, idx[i])
    acc_PC_compile = np.concatenate((acc_PC_compile, acc_tmp), axis = 0)

# plot accuracy as a function of PCs
plt.plot(np.linspace(1,np.size(acc_PC_compile,0), np.size(acc_PC_compile,0)), acc_PC_compile, 'k')
plt.title('Classification accuracy')
plt.xlabel('# of PCs included')
plt.ylabel('Accuracy (%)')
plt.ylim(30, 100)
plt.xlim(-5, 105)

## Permutation testing for each number of PCs

In [0]:
def classification_RF_shuffle_PCs(train_data, train_lab_idx_shuffle, test_data, test_lab_idx, loadings, num_pc):

    load_train = np.matmul(train_data, loadings[:,:num_pc])
    
    rf_1 = RandomForestClassifier()
    rng.shuffle(train_lab_idx_shuffle)
    rf_1.fit(load_train, train_lab_idx_shuffle)
    
    load_test = np.matmul(test_data, loadings[:,:num_pc])
    
    # Predict unseen data
    predictions_1 = rf_1.predict(load_test)
    test_lab_idx = np.asarray(test.iloc[:].label)
    
    # compare predictions with test labels and compute accuracy
    result = predictions_1 - test_lab_idx
    result_binary = np.argwhere(result == 0)
    correct = np.size(result_binary,0)
    acc = correct/test_lab_idx.shape[0]*100
    return acc

def iteration_permute_PCs(train_data, train_lab_idx_shuffle, test_data, test_lab_idx, loadings, num_pc):
    
    acc_shuffle = []
    for i in range(100):
        acc_tmp = dask.delayed(classification_RF_shuffle_PCs)(train_data, train_lab_idx_shuffle, test_data, test_lab_idx, loadings, num_pc)
        acc_shuffle.append(acc_tmp)
    
    # compute values
    acc_shuffle = dask.compute(*acc_shuffle)
    acc_shuffle = np.array([float(num) for num in acc_shuffle])
    clear_output(wait = True)
    return acc_shuffle

# def iteration_shuffle_PCs(train_data, train_lab_idx, test_data, test_lab_idx, loadings, pc_idx):
    
#     acc_PC = []
#     for i in pc_idx:
#         acc_tmp = dask.delayed(iteration_permute_PCs)(train_data, train_lab_idx_shuffle, test_data, test_lab_idx, loadings, i)
#         acc_PC.append(acc_tmp)
    
#     # compute values
#     acc_PC = dask.compute(*acc_PC)
#     clear_output(wait = True)
#     # acc_PC = np.array([float(num) for num in acc_PC])
#     return acc_PC

In [0]:
%%time

idx = np.linspace(1,100,100, dtype = 'int')
acc_shuffle_PC_compile = []
for i in idx:
    acc_tmp = iteration_permute_PCs(train_data, train_lab_idx_shuffle, test_data, test_lab_idx, loadings, i)
    acc_shuffle_PC_compile.append(acc_tmp)


## Compare classification accuracy to chance level for each number of PCs

In [0]:
# plot accuracy as a function of PCs

# accuracy as a function of PCs
plt.plot(np.linspace(1,np.size(acc_PC_compile,0), np.size(acc_PC_compile,0)), acc_PC_compile, 'k')

# corresponding chance level
p_value_compile = []
for i in range(len(acc_shuffle_PC_compile)):
    b = plt.boxplot(acc_shuffle_PC_compile[i], 'b', positions = [i+1], flierprops = {'marker': '.', 'markersize': 5})

    # permutation testing
    null_above_value = (acc_shuffle_PC_compile[i] > acc_PC_compile[i]).sum()
    p_value_compile.append(null_above_value/np.size(acc_shuffle_PC_compile[i],0))

# visualize statistical results
p_value_compile = np.array([float(num) for num in p_value_compile])
p_value_adj = false_discovery_control(p_value_compile)   

for i in range(len(p_value_adj)): 
    if p_value_adj[i] < 0.05:
        plt.scatter(i, 95, s = 10, c = 'r', marker = "*")

plt.title('Classification accuracy')
plt.xlabel('# of PCs included')
plt.ylabel('Accuracy (%)')
plt.ylim(30, 100)
plt.xlim(-5, 105)
p_value_compile

## Feature interpretation

### Visualize spatial PC maps to identify features that contribute to data variance and classification

In [0]:
f, ax = plt.subplots(5,4)
lim = 20
for ipc in range(lim):

    PC_map = loadings[:,ipc]
    PC_map = np.reshape(PC_map, (128,128))
    clim = max(abs(np.min(PC_map)), abs(np.max(PC_map)))

    idx1 = np.floor(ipc/4).astype(int)
    idx2 = np.fmod(ipc,4)
    ax[idx1, idx2].axis('off')
    ax[idx1, idx2].imshow(PC_map, cmap = "magma", vmin = -clim, vmax = clim)
    ax[idx1, idx2].set_title(f'PC# {ipc +1}', fontsize = 10)
    plt.tight_layout()

#### Here we can see that for each of the top 20 PC's there are brain regions are more strongly weighted than others (in either the positive or negative direction). It is important to note that it does not appear that each image used in this dataset came from the same location/slice within the brain. We do not know if certain slices are over- or under-represented in different AD classes and therefore cannot say with certainty whether specific anatomical structures drive different stages of AD. Nevertheless,